In [1]:
%pip install  --force-reinstall --upgrade --no-deps  git+https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git@e18b4dbe5a1cf686f6f7fa7bb8888928af545c47

  Cloning https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git (to revision e18b4dbe5a1cf686f6f7fa7bb8888928af545c47) to /tmp/pip-req-build-2_ipvxvn
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git /tmp/pip-req-build-2_ipvxvn
  Running command git rev-parse -q --verify 'sha^e18b4dbe5a1cf686f6f7fa7bb8888928af545c47'
  Running command git fetch -q https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git e18b4dbe5a1cf686f6f7fa7bb8888928af545c47
  Running command git checkout -q e18b4dbe5a1cf686f6f7fa7bb8888928af545c47
  Resolved https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git to commit e18b4dbe5a1cf686f6f7fa7bb8888928af545c47
  Preparing metadata (setup.py) ... done
  Created wheel for classifier-free-guidance-pytorch: filename=classifier_free_guidance_pytorch-0.6.2-py3-none-any.whl size=1372777 sha256=776499dd947811633720ed9254d20c0b5811e1d180488e41441e0cf37ca154d6
  St

In [2]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-h3rnlkvu
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-h3rnlkvu
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit b5e74674973de4a97431561d7723d9b29629a695
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pathlib import Path 
import gc    
import torch
import os
import torch  
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)   


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,   
       # codebook_size =  2048, 
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8,
    
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2
    ).to("cuda")     

In [ ]:
%cd /root/text-to-mesh

/root/text-to-mesh


In [ ]:
dataset = MeshDataset.load("./1.55k.npz") 
labels = list(set(item['texts'] for item in dataset.data))
dataset.generate_codes(autoencoder,150)

[MeshDataset] Loaded 1000 entrys
[MeshDataset] Created from 1000 entrys


  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:03<00:00,  2.27it/s]

[MeshDataset] Generated codes for 1000 entrys


In [ ]:

torch.cuda.empty_cache()
gc.collect()  
  
max_length = max(len(d['faces']) for d in dataset.data) * 6
print("max_length", max_length)
transformer = MeshTransformer(
    autoencoder,
    dim = 768,
    coarse_pre_gateloop_depth =2,  
    fine_pre_gateloop_depth= 2, 
    attn_depth = 12,  
    attn_heads = 12, 
    fine_cross_attend_text = True,
    text_cond_with_film = False,
    cross_attn_num_mem_kv = 4,
    num_sos_tokens = 1, 
    dropout  = 0.0,
    max_seq_len = max_length + 6, 
    fine_attn_depth = 2,
    condition_on_text = True, 
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", 
    text_condition_cond_drop_prob = 0.0, 
).to("cpu")  
 

max_length 9312


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
transformer.conditioner.text_models[0].model.to("cuda")
transformer = transformer.to("cuda")

dataset.embed_texts(transformer,1)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
 
torch.cuda.empty_cache()
gc.collect()  
 
batch_size =1
grad_accum_every =4     
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=grad_accum_every,num_train_steps=100, dataset = dataset, 
                                  accelerator_kwargs = {"mixed_precision" : "fp16"}, optimizer_kwargs = { "eps": 1e-7} , 
                                 learning_rate = 1e-3, batch_size=batch_size ,checkpoint_every_epoch = 25) 
loss = trainer.train(35, stop_at_loss = 0.00005)   
 